# I - Setup
This notebook will set everything up for the tutorial. This notebook assumes that nothing has been set up previously and will create everything from scratch. The necessary steps are broken up into the following sections:
* [Install tools and dependencies](#section1)
* [Azure account login](#section2)
* [Setup](#section3)
* [Create Azure resources](#section4)
* [Batch Shipyard Configuration](#section5)
* [Create Azure Batch Pool](#section6)
* [View Resources on Azure Portal or Batch Labs](#section7)

<a id='section1'></a>

## Install tools and dependencies
Install Batch Shipyard and Azure CLI 2.0

In [ ]:
!git clone https://github.com/Azure/batch-shipyard.git

Normally you would use `install.sh` or `install.cmd` helper scripts to install, but due to the Notebook environment, we will instead install with the `requirements.txt` file directly.

In [ ]:
!pip install -r batch-shipyard/requirements.txt

We will pre-download the Nvidia dependencies now to avoid the interactive license agreement prompt. If running outside of the Notebook environment then this step is not necessary.

In [ ]:
!wget -O $HOME/batch-shipyard/resources/nvidia-docker.deb https://github.com/NVIDIA/nvidia-docker/releases/download/v1.0.1/nvidia-docker_1.0.1-1_amd64.deb
!wget -O $HOME/batch-shipyard/resources/nvidia-driver.run http://us.download.nvidia.com/XFree86/Linux-x86_64/375.51/NVIDIA-Linux-x86_64-375.51.run

Azure CLI 2.0 will also be installed to help us in provisioning Azure Batch and Storage accounts.

In [ ]:
!pip install -I azure-cli

We'll create an alias for invoking Batch Shipyard which points to a `config` directory which will hold our json config files (this directory will be created at a later step).

In [ ]:
%alias shipyard SHIPYARD_CONFIGDIR=config python $HOME/batch-shipyard/shipyard.py %l

Check that everything is working:

In [ ]:
shipyard

<a id='section2'></a>

## Azure account login
The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser if you do not specify a username and password or your organization requires multi-factor authentication.

In [ ]:
# login using browser device code
!az login -o table

# login using username and password (this may not work if your account requires multi-factor auth, use the above method instead)
#USERNAME = "account@domain.com"    # replace with your username
#PASSWORD = "supersecretpassword"  # replace with your password
#!az login -u $USERNAME -p $PASSWORD -o table

If you have multiple subscriptions you can select the one you need with the command below.

In [ ]:
selected_subscription = '"MY SUBSCRIPTION NAME"' # Replace with the name of your subscription
!az account set --subscription $selected_subscription

**Note:** If you cannot login with the Azure CLI, you can create Azure Batch and Storage accounts on the [Azure Portal](https://portal.azure.com).
- [Instructions for creating an Azure Batch Account](https://docs.microsoft.com/en-us/azure/batch/batch-account-create-portal#batch-service-mode)
- [Instructions for creating an Azure Storage Account](https://docs.microsoft.com/en-us/azure/storage/storage-create-storage-account#create-a-storage-account) (You can create an "Auto Storage" account at the same time as your Batch Account on the portal instead)

Please pay attention to special instructions regarding Azure Portal created accounts below.

<a id='section3'></a>

## Setup
Below are the various name definitions for the resources needed to run batch jobs.

**Note:** If you manually created your accounts on the Azure Portal, you will need to modify `GROUP_NAME`, `BATCH_ACCOUNT_NAME` and `STORAGE_ACCOUNT_NAME` accordingly.

In [ ]:
import json
import os
import uuid

def write_json_to_file(json_dict, filename):
    """ Simple function to write JSON dictionaries to files
    """
    with open(filename, 'w') as outfile:
        json.dump(json_dict, outfile)

LOCATION = 'southcentralus' # We are setting everything up in South Central US
                            # Be aware that you need to set things up in a region that has GPU VMs (N-Series)

IMAGE_NAME = "masalvar/cntkcifar" # Custom CNTK image

# Please do not modify below unless you created your accounts on the Azure Portal

short_uuid = str(uuid.uuid4())[:8]

GROUP_NAME = "batchcntk{uuid}rg".format(uuid=short_uuid)
BATCH_ACCOUNT_NAME = "batchcntk{uuid}ba".format(uuid=short_uuid)
STORAGE_ACCOUNT_NAME = "batchcntk{uuid}str".format(uuid=short_uuid)
STORAGE_ALIAS = "mystorageaccount"
STORAGE_ENDPOINT = "core.windows.net"

##### This cell is for recovery purposes only. There is no need to run this for normal executions:

GROUP_NAME = ""
BATCH_ACCOUNT_NAME = ""
STORAGE_ACCOUNT_NAME = ""
batch_account_key = ""
batch_service_url = ""
storage_account_key = ""

<a id='section4'></a>

## Create Azure resources

### Create Resource Group
Azure encourages the use of resource groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the Resource Group.

In [ ]:
%%time
!az group create -n $GROUP_NAME -l $LOCATION -o table

### Create Batch and Storage accounts
Here we simply crate the Batch and Storage accounts. Once we have created the accounts we can the use the Azure CLI to query them and obtain the **batch_account_key**, **batch_service_url** and **storage_account_key** which we will need for our Batch Shipyard configuration files later.

In [ ]:
json_data = !az storage account create -l $LOCATION -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME --sku Standard_LRS
print('Storage account provisioning state: {}'.format(json.loads(''.join(json_data))['provisioningState']))

In [ ]:
json_data = !az batch account create -l $LOCATION -n $BATCH_ACCOUNT_NAME -g $GROUP_NAME --storage-account $STORAGE_ACCOUNT_NAME
print('Batch account provisioning state: {}'.format(json.loads(''.join(json_data))['provisioningState']))

Next we retrieve the **batch_account_key**, **batch_service_url** and **storage_account_key** which we will need for the Batch Shipyard configuration files further down.

**Note:** If you created your Batch and Storage accounts in the Azure Portal, you will need to retrieve your keys in the Portal. Then set `batch_account_key`, `batch_service_url`, and `storage_account_key` to their appropriate values instead of the Azure CLI callouts.

In [ ]:
json_data = !az batch account keys list -n $BATCH_ACCOUNT_NAME -g $GROUP_NAME
batch_account_key = json.loads(''.join(json_data))['primary']

In [ ]:
json_data = !az batch account list -g $GROUP_NAME
batch_service_url = 'https://'+json.loads(''.join(json_data))[0]['accountEndpoint']

In [ ]:
json_data = !az storage account keys list -n $STORAGE_ACCOUNT_NAME -g $GROUP_NAME
storage_account_key = json.loads(''.join(json_data))[0]['value']

Save credentials required for other notebooks.

In [ ]:
account_information = {
    "storage_account_key": storage_account_key,
    "storage_account_name": STORAGE_ACCOUNT_NAME,
    "resource_group": GROUP_NAME,
    "location": LOCATION,
}
write_json_to_file(account_information, 'account_information.json')

<a id='section5'></a>

## Batch Shiyard configuration
In order to execute a job on Batch Shipyard you need a minimum of four configuration files. We will set three of them here and leave the job one for later.
* [credentials](#credentials)
* [configuration](#configuration)
* [pool](#pool)


<a id='credentials'></a>

### Credentials
Here we define all the credentials necessary for Batch Shipyard to connect to Azure for resource provisioning and executing our jobs.

In [ ]:
credentials = {
    "credentials": {
        "batch": {
            "account_key": batch_account_key,
            "account_service_url": batch_service_url
        },
        "storage": {
            STORAGE_ALIAS : {
                    "account": STORAGE_ACCOUNT_NAME,
                    "account_key": storage_account_key,
                    "endpoint": STORAGE_ENDPOINT
            }
        }
    }
}

<a id='configuration'></a>

### Configuration
The config mainly contains the configuration for Batch Shipyard. Here we simply define the storage alias that Batch Shipyard should use as well as the Docker image to use.

In [ ]:
config = {
    "batch_shipyard": {
        "storage_account_settings": STORAGE_ALIAS
    },
    "global_resources": {
        "docker_images": [
            IMAGE_NAME
        ]
    }
}

<a id='pool'></a>

### Pool
This is where we define the properties of the compute pool we wish to create. The configuration below creates a pool that is made up of a three NC6 VM running Ubuntu. If you wish to run a job that uses GPUs then you need to use a VM from the NC series.

In [ ]:
POOL_ID = 'gpupool'

pool = {
    "pool_specification": {
        "id": POOL_ID,
        "vm_size": "STANDARD_NC6",
        "vm_count": 3,
        "publisher": "Canonical",
        "offer": "UbuntuServer",
        "sku": "16.04-LTS",
        "ssh": {
            "username": "docker"
        },
        "reboot_on_start_task_failed": False,
        "block_until_all_global_resources_loaded": True,
    }
}

In [ ]:
!mkdir -p config # Create config file directory where we will store all our Batch Shipyard configuration files

In [ ]:
write_json_to_file(credentials, os.path.join('config', 'credentials.json'))

In [ ]:
write_json_to_file(config, os.path.join('config', 'config.json'))

In [ ]:
write_json_to_file(pool, os.path.join('config', 'pool.json'))

<a id='section6'></a>

## Create Azure Batch Pool
Before we do anything we need to create the pool for Batch Shipyard jobs to run on. This can take a little bit of time so be patient.

**Note:** As soon as one node enters `ComputeNodeState.idle` you can proceed you can proceed to the next notebook.

In [ ]:
shipyard pool add

Once the pool is created we can confirm everything by running the command below.

In [ ]:
shipyard pool listnodes

Please run the following cell and copy/paste the information output from the cell into a scratch space (e.g., Notepad), just in case for recovery purposes:

In [ ]:
print('GROUP_NAME = "{}"'.format(GROUP_NAME))
print('BATCH_ACCOUNT_NAME = "{}"'.format(BATCH_ACCOUNT_NAME))
print('STORAGE_ACCOUNT_NAME = "{}"'.format(STORAGE_ACCOUNT_NAME))
print('batch_account_key = "{}"'.format(batch_account_key))
print('batch_service_url = "{}"'.format(batch_service_url))
print('storage_account_key = "{}"'.format(storage_account_key))

<a id='section7'></a>
## View Resources on Azure Portal or Batch Labs

If you want a graphical interface for your Azure Batch resources, you can view them on the [Azure Portal](https://portal.azure.com) or on [Batch Labs](https://github.com/Azure/BatchLabs).

[Next notebook: Single GPU Training](02_Single_GPU_Training.ipynb)